In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [4]:
import pandas as pd

metadata_path = "/kaggle/input/abo-small/metadata/images.csv"
df = pd.read_csv(metadata_path)
df.head()
print("Total images:", len(df))

Total images: 398212


In [5]:
import json

# Loading the listings
with open("/kaggle/input/abo-product-metadata/combined_listings.json", "r") as f:
    data = json.load(f)

print(f"Total entries: {len(data)}")

# Printing a sample listing
sample_id = list(data.keys())[0]
print("Sample image_id:", sample_id)
print("Associated product data:", data[sample_id])

Total entries: 398170
Sample image_id: 81iZlv3bjpL
Associated product data: {'brand': [{'language_tag': 'en_GB', 'value': 'find.'}], 'item_id': 'B06X9WQGQP', 'item_name': [{'language_tag': 'en_GB', 'value': 'find. Women’s Ari Heeled Closed-Toe Heels'}], 'model_name': [{'language_tag': 'en_GB', 'value': 'Ari Heeled'}], 'model_year': [{'value': 2017}], 'product_type': [{'value': 'SHOES'}], 'style': [{'language_tag': 'en_GB', 'value': 'Closed-Toe Pumps'}], 'main_image_id': '81iZlv3bjpL', 'other_image_id': ['91mIRxgziUL', '91eqBkW06wL', 'A1BHZSKNbkL'], 'country': 'GB', 'marketplace': 'Amazon', 'domain_name': 'amazon.co.uk', 'node': [{'node_id': 1769851031, 'node_name': "/Categories/Shoes/Women's Shoes/Court Shoes"}]}


In [6]:
# Printing a sample listing
sample_id = list(data.keys())[5]
print("Sample image_id:", sample_id)
print("Associated product data:", data[sample_id])

Sample image_id: 51Fqps5k9YL
Associated product data: {'item_dimensions': {'height': {'normalized_value': {'unit': 'inches', 'value': 0.87}, 'unit': 'inches', 'value': 0.87}, 'length': {'normalized_value': {'unit': 'inches', 'value': 22}, 'unit': 'inches', 'value': 22}, 'width': {'normalized_value': {'unit': 'inches', 'value': 0.9}, 'unit': 'inches', 'value': 0.9}}, 'brand': [{'language_tag': 'tr_TR', 'value': 'AmazonBasics'}], 'color': [{'language_tag': 'tr_TR', 'value': 'Weiße Pulverbeschichtung'}], 'item_id': 'B01MS9NPOE', 'item_name': [{'language_tag': 'tr_TR', 'value': 'AmazonBasics – Avrupa çekmece rayları, alt tarafa sabitlemek için, AB-5013'}], 'model_number': [{'value': 'AB-5013'}], 'product_type': [{'value': 'HARDWARE'}], 'main_image_id': '619y9YG9cnL', 'other_image_id': ['51Fqps5k9YL', '61b6xUQutSL', '61C35DEYVkL'], 'country': 'TR', 'marketplace': 'Amazon', 'domain_name': 'amazon.com.tr', 'node': [{'node_id': 12707320031, 'node_name': '/Kategoriler/Hırdavat/Etajer Ekipmanlar

In [7]:
def extract_metadata(data):
    metadata_lines = []

    def get_en_value(field):
        """Helper to extract English value from a language-tagged list."""
        return next((entry.get('value') for entry in data.get(field, [])
                     if entry.get('language_tag', '').startswith('en') and entry.get('value')), None)


    item_name = get_en_value('item_name')
    if item_name:
        metadata_lines.append(f"Item Name: {item_name}")

    model_name = get_en_value('model_name')
    if model_name:
        metadata_lines.append(f"Model Name: {model_name}")
        
    brand = get_en_value('brand')
    if brand:
        metadata_lines.append(f"Brand: {brand}")


    product_type = data.get('product_type', [{}])[0].get('value')
    if product_type:
        metadata_lines.append(f"Product Type: {product_type}")

    style = get_en_value('style')
    if style:
        metadata_lines.append(f"Style: {style}")

    color = get_en_value('color')
    if color:
        metadata_lines.append(f"Color: {color}")

    # Weight
    weight_data = data.get('item_weight', [{}])[0].get('normalized_value', {})
    if weight_data:
        weight = weight_data.get('value')
        unit = weight_data.get('unit')
        if weight is not None and unit:
            metadata_lines.append(f"Item Weight: {weight} {unit}")

    # Dimensions
    dimensions = data.get('item_dimensions')
    if dimensions:
        height = dimensions.get('height', {}).get('value')
        length = dimensions.get('length', {}).get('value')
        width = dimensions.get('width', {}).get('value')
        unit = dimensions.get('height', {}).get('unit')
        if None not in (height, length, width, unit):
            metadata_lines.append(f"Item Dimensions (H x L x W): {height} x {length} x {width} {unit}")

    # Bullet Points (only English)
    bullet_points = [
        bp['value'] for bp in data.get('bullet_point', [])
        if bp.get('language_tag', '').startswith('en') and bp.get('value')
    ]
    if bullet_points:
        metadata_lines.append("Bullet Points: " + "; ".join(bullet_points))


    item_keywords = get_en_value('item_keywords')
    if item_keywords:
        metadata_lines.append(f"Item Keywords: {item_keywords}")

    return "\n".join(metadata_lines)

In [8]:
print(extract_metadata(data[sample_id]))

Product Type: HARDWARE
Item Dimensions (H x L x W): 0.87 x 22 x 0.9 inches


In [9]:
import json
import random
from collections import defaultdict


print(f"Total entries in original data: {len(data)}")

# Helper to check if an entry is in English
def is_english(entry):
    for field in ['brand', 'bullet_point', 'color', 'item_name', 'style', 'item_keywords']:
        if field in entry:
            values = entry[field]
            if not all(v.get('language_tag', '').startswith('en') for v in values):
                return False
    return True

# Filter to English-only entries
english_entries = {
    img_id: info for img_id, info in data.items() if is_english(info)
}

print(f"Entries after English filtering: {len(english_entries)}")

# Group by product type
product_type_groups = defaultdict(list)
for img_id, info in english_entries.items():
    product_type = info.get('product_type', [{}])[0].get('value')
    if product_type:
        product_type_groups[product_type].append((img_id, info))

# Determine equal sample size per product type
num_types = len(product_type_groups)
target_total = 30000
per_type_quota = target_total // num_types
sampled_data = {}

print(f"Sampling approx {per_type_quota} entries per product type...")

# First round of balanced sampling
for ptype, items in product_type_groups.items():
    sample_count = min(per_type_quota, len(items))
    sampled = random.sample(items, sample_count)
    for img_id, info in sampled:
        sampled_data[img_id] = info

# Fill remaining with random from types that had more than quota
remaining = target_total - len(sampled_data)
overflow_pool = []

for ptype, items in product_type_groups.items():
    available = [item for item in items if item[0] not in sampled_data]
    overflow_pool.extend(available)

if remaining > 0 and overflow_pool:
    additional_samples = random.sample(overflow_pool, min(remaining, len(overflow_pool)))
    for img_id, info in additional_samples:
        sampled_data[img_id] = info

print(f"Final sampled size: {len(sampled_data)}")

# Save the sampled data
with open("sampled_english_equal_product_types.json", "w") as f_out:
    json.dump(sampled_data, f_out)

print("Saved to 'sampled_english_equal_product_types.json'")


Total entries in original data: 398170
Entries after English filtering: 216546
Sampling approx 67 entries per product type...
Final sampled size: 30000
Saved to 'sampled_english_equal_product_types.json'


In [10]:
image_base_path = "/kaggle/input/abo-small/small"
metadata_path = "/kaggle/input/abo-small/metadata/images.csv"
metadata_df = pd.read_csv(metadata_path)

In [11]:
!pip install google-generativeai tqdm --quiet

In [12]:
sample_key, val = None, None
for key, value in sampled_data.items():
    sample_key = key
    val = value
    break

In [13]:
metadata_df = metadata_df.set_index("image_id")

# Then retrieve path directly
path = metadata_df.at[val.get("main_image_id"), "path"]
print("Path:", path)

Path: c0/c0749f20.jpg


In [14]:
def extract_json(output):
    try:
        start = output.find('[')
        end = output.rfind(']')
        json_str = output[start:end+1]

        data = json.loads(json_str)

        # Check format
        if not isinstance(data, list):
            raise ValueError("The response should be a list of Q&A pairs.")
        for item in data:
            if not isinstance(item, dict):
                raise ValueError("Each item in the list should be a dictionary.")
            if "question" not in item or "answer" not in item:
                raise ValueError("Each dictionary should contain 'question' and 'answer' keys.")
            if not isinstance(item["question"], str) or not isinstance(item["answer"], str):
                raise ValueError("Both 'question' and 'answer' should be strings.")

        return data

    except:
        return None

In [15]:
extract_metadata(val)

"Item Name: Fusefit Men's Sapphire Red Running Shoes-8 UK (42 EU) (9 US) (FFR-410_8)\nModel Name: Sapphire\nBrand: Fusefit\nProduct Type: SHOES\nStyle: Running Shoes\nColor: Red\nBullet Points: Outer Material: Mesh; Closure Type: Slip On; Toe Style: Round Toe; Warranty Type: Manufacturer; Warranty Description: 30 days\nItem Keywords: shoes"

In [15]:
import google.generativeai as genai
from PIL import Image
import os

genai.configure(api_key="your_api_key")

model = genai.GenerativeModel("gemini-1.5-flash")

image_path = os.path.join(image_base_path, path)
image = Image.open(image_path)



prompt = '''You are shown a product image along with its metadata. Use the metadata only for *contextual understanding*, not for answering. Your task is to generate a set of **4–6 diverse, image-answerable, and challenging questions**, where **each answer is a single word** (e.g., "blue", "five", "yes", "sneakers").

These questions must require **careful visual inspection**, **fine-grained detail recognition**, or **higher-level visual reasoning**, but must be **fully answerable using only the image itself**. The metadata is provided to help disambiguate what the image might contain — do not base your questions on metadata-only information.

Question Requirements:
- Must be **clearly grounded in visible evidence** in the image
- Must be **unambiguous**, **non-subjective**, and **non-trivial**
- Each answer must be a **single word** (e.g., noun, color, number, "yes"/"no")

Coverage:
Include **at least one** question that requires:
1. Small detail recognition 
2. Fine-grained categorization 
3. Challenging reasoning or fine-grained counting

Also consider:
- Material/texture recognition
- Color attribute specificity
- Brand/logo identification
- Shape or structure understanding
- Functional yes/no questions

Difficulty:
- Mix easy, moderate, and hard — with emphasis on **moderate to hard**
- Avoid obvious or metadata-derived questions
- Avoid subjective or vague terms like "modern" or "comfortable"

Output Format:
Return only a list of dictionaries in this exact format:

[
    {
        "question": "What specific type of product is shown?",
        "answer": "Backpack"
    },
    {
        "question": "What material is most visible on the product?",
        "answer": "Leather"
    },
    {
        "question": "What color is the zipper?",
        "answer": "Silver"
    },
    {
        "question": "Is the logo present on the front side?",
        "answer": "Yes"
    },
    {
        "question": "How many pockets are visible?",
        "answer": "Three"
    }
]
''' + f"Image Metadata: {extract_metadata(val)}"

print("Generating response")
response = model.generate_content([image, prompt])
text = response.text

print(text)
print(extract_json(text))

Generating response
```json
[
  {
    "question": "What is the toe style of the shoe?",
    "answer": "Pointed"
  },
  {
    "question": "What color is the buckle?",
    "answer": "Gold"
  },
  {
    "question": "What is the primary material of the shoe upper?",
    "answer": "PU"
  },
  {
    "question": "Is there a visible heel?",
    "answer": "No"
  },
  {
    "question": "How many straps are visible on the shoe?",
    "answer": "One"
  },
  {
    "question": "What is the closure type of the shoes?",
    "answer": "Slip-on"
  }
]
```

[{'question': 'What is the toe style of the shoe?', 'answer': 'Pointed'}, {'question': 'What color is the buckle?', 'answer': 'Gold'}, {'question': 'What is the primary material of the shoe upper?', 'answer': 'PU'}, {'question': 'Is there a visible heel?', 'answer': 'No'}, {'question': 'How many straps are visible on the shoe?', 'answer': 'One'}, {'question': 'What is the closure type of the shoes?', 'answer': 'Slip-on'}]


In [21]:
import os
import json
import time
import pandas as pd
from PIL import Image
import google.generativeai as genai
from tqdm import tqdm

# === CONFIGURATION ===
API_KEY = "your_api_key"
LISTINGS_JSON_PATH = "/kaggle/input/product-listings/product_listings.json"
OUTPUT_JSON = "vqa_results_19500_to_20000.json"

SLEEP_TIME = 5  # in seconds

START_INDEX = 19500
END_INDEX = 20000

SLEEP_TIME = 5  # in seconds


keys = list(data.keys())
data_subset = {k: data[k] for k in keys[START_INDEX:END_INDEX]}

# === Setup Gemini ===
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")


# === Load listings JSON ===
with open(LISTINGS_JSON_PATH, "r") as f:
    data = json.load(f)


all_results = {}

# === Iterate through listings ===
count = 0
for key, val in tqdm(data_subset.items()):

    try:
        image_id = val.get("main_image_id")
        if not image_id:
            continue

        # Retrieve image path
        path = metadata_df.at[image_id, "path"]
        image_path = os.path.join(image_base_path, path)
        if not os.path.exists(image_path):
            continue

        # Open image
        image = Image.open(image_path)

        # Compose prompt
        prompt = '''You are shown a product image along with its metadata. Use the metadata only for *contextual understanding*, not for answering. Your task is to generate a set of **4–6 diverse, image-answerable, and challenging questions**, where **each answer is a single word** (e.g., "blue", "five", "yes", "sneakers").

These questions must require **careful visual inspection**, **fine-grained detail recognition**, or **higher-level visual reasoning**, but must be **fully answerable using only the image itself**. The metadata is provided to help disambiguate what the image might contain — do not base your questions on metadata-only information.

Question Requirements:
- Must be **clearly grounded in visible evidence** in the image
- Must be **unambiguous**, **non-subjective**, and **non-trivial**
- Each answer must be a **single word**

Coverage:
Include **at least one** question that requires:
1. Small detail recognition 
2. Fine-grained categorization 
3. Challenging reasoning or fine-grained counting

Also consider:
- Material/texture recognition
- Color attribute specificity
- Brand/logo identification
- Shape or structure understanding
- Functional yes/no questions
- Any other visible aspects of the image

Difficulty:
- Mix easy, moderate, and hard — with emphasis on **moderate to hard**
- Avoid obvious or metadata-derived questions
- Avoid subjective or vague terms like "modern" or "comfortable"

Output Format:
Return only a list of dictionaries in this exact format:

[
    {
        "question": "What specific type of product is shown?",
        "answer": "Backpack"
    },
    {
        "question": "What material is most visible on the product?",
        "answer": "Leather"
    },
    {
        "question": "What color is the zipper?",
        "answer": "Silver"
    },
    {
        "question": "Is the logo present on the front side?",
        "answer": "Yes"
    },
    {
        "question": "How many pockets are visible?",
        "answer": "Three"
    }
]
''' + f"\nImage Metadata: {extract_metadata(val)}"

        print(f"Generating response for {key}...")
        response = model.generate_content([image, prompt])
        qa_data = response.text.strip()
        final_qa = extract_json(qa_data)

        all_results[key] = {
            "image_id": image_id,
            "qa_data": final_qa
        }

        # Save intermediate result
        with open(OUTPUT_JSON, "w") as f:
            json.dump(all_results, f, indent=2)

        count += 1
        time.sleep(SLEEP_TIME)

    except Exception as e:
        print(f"[ERROR] Failed for {key}: {e}")
        time.sleep(SLEEP_TIME * 2)  # longer pause after error
        continue

print(f"\n✅ Completed {count} listings. Results saved to {OUTPUT_JSON}")


  0%|          | 0/500 [00:00<?, ?it/s]

Generating response for 717lPoaPRUL...


  0%|          | 1/500 [00:07<58:27,  7.03s/it]

Generating response for 81Ss+9KGOPL...


  0%|          | 2/500 [00:13<56:53,  6.85s/it]

...

Generating response for 61IASG4JkML...


100%|█████████▉| 499/500 [56:46<00:06,  6.86s/it]

Generating response for 910u8oMh3sL...


100%|██████████| 500/500 [56:53<00:00,  6.83s/it]


✅ Completed 500 listings. Results saved to vqa_results_19500_to_20000.json
